<a href="https://colab.research.google.com/github/yunnsss/Fog_Forecast/blob/main/Stacking_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensemble 모델로 안개 데이터 예측

## 참고 : https://dacon.io/codeshare/4651

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost
!pip install keras==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.12.0 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier


from sklearn.model_selection import train_test_split

import imblearn
from imblearn.over_sampling import SMOTE

from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.metrics import f1_score, mean_absolute_error

from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.losses import SparseCategoricalCrossentropy
from tqdm.notebook import tqdm

## 데이터 불러오기

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2024날씨마루콘테스트/train_FE.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2024날씨마루콘테스트/test_FE.csv')

In [ ]:
test = test.fillna(0)

In [ ]:
# 필요없는 열 제거
train = train.drop(labels=['sin_week', 'cos_week',
                                'sin_year', 'cos_year'
                                ], axis = 1)

test = test.drop(labels=['sin_week', 'cos_week',
                                'sin_year', 'cos_year'
                                ], axis = 1)

In [ ]:
# stn_id의 첫 번째 글자 추출
train['group'] = train['stn_id'].str[0]
test['group'] = test['stn_id'].str[0]

# 각 글자별로 데이터프레임 분리
stn_A = train[train['group'] == 'A']
stn_B = train[train['group'] == 'B']
stn_C = train[train['group'] == 'C']
stn_D = train[train['group'] == 'D']
stn_E = train[train['group'] == 'E']

# 테스트 데이터프레임 분리
test_stn_A = test[test['group'] == 'A']
test_stn_B = test[test['group'] == 'B']
test_stn_C = test[test['group'] == 'C']
test_stn_D = test[test['group'] == 'D']
test_stn_E = test[test['group'] == 'E']

## Model

#### split


In [ ]:
from sklearn.model_selection import train_test_split

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = stn_A['class']

In [ ]:
stn_A = stn_A.drop(labels=['year','month','day','time','minute', 'season', 'stn_id', 'group', 'datetime', 'vis1','class'], axis=1)

In [ ]:
# train data를 8:2로 train data와 valid data로 분리
X_train, X_val, y_train, y_val = train_test_split(stn_A, target,
                                                      test_size=0.2,
                                                      random_state=11)

#### smote

In [ ]:
# SMOTE oversampling
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

#### 스태킹 앙상블

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
#개별 ML 모델 생성
knn_clf = KNeighborsClassifier(n_neighbors = 4)
rf_clf = RandomForestClassifier(n_estimators = 100, random_state = 0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators = 100)

# 스태킹으로 만들어진 데이터 세트를 학습, 예측할 최종 모델
lr_final = LogisticRegression(C=10)

In [ ]:
#개별 모델 학습
knn_clf.fit(X_train_resampled, y_train_resampled)

KNeighborsClassifier(n_neighbors=4)

In [ ]:
rf_clf.fit(X_train_resampled, y_train_resampled)

KeyboardInterrupt: 

In [ ]:
dt_clf.fit(X_train_resampled, y_train_resampled)

In [ ]:
ada_clf.fit(X_train_resampled, y_train_resampled)

In [ ]:
# 학습된 개별 모델들이 각자 반환하는 예측 데이터 세트를 생성하고 개별 모델의 정확도 측정
knn_pred = knn_clf.predict(X_val)
rf_pred = rf_clf.predict(X_val)
dt_pred = dt_clf.predict(X_val)
ada_pred = ada_clf.predict(X_val)

In [ ]:
print('KNN 정확도: {0:.4f}'.format(accuracy_score(y_val, knn_pred)))
print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy_score(y_val, rf_pred)))
print('결정 트리 정확도: {0:.4f}'.format(accuracy_score(y_val, dt_pred)))
print('에이다부스트 정확도: {0:.4f} :'.format(accuracy_score(y_val, ada_pred)))

In [ ]:
# 기반 모델의 예측 결과를 스태킹
stacked = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(stacked.shape)

# transpose를 이용, 행과 열의 위치를 교환
stacked = np.transpose(stacked)
print(stacked.shape)

In [ ]:
# 메타 모델은 기반모델의 예측결과를 기반으로 학습
lr_final.fit(stacked, y_val)

In [ ]:
# 예측 및 성능 평가
y_pred = lr_final.predict(stacked)

In [ ]:
mae = mean_absolute_error(y_val, y_pred)
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='weighted')

print(f"스태킹 앙상블 모델의 MAE: {mae}")
print(f"스태킹 앙상블 모델의 정확도: {accuracy}")
print(f"스태킹 앙상블 모델의 F1 점수: {f1}")

In [ ]:
# 각 값의 빈도 계산
unique_values, counts = np.unique(y_pred, return_counts=True)
print(dict(zip(unique_values, counts)))

# y_pred_classes = y_pred.argmax(axis=-1) + 1

##############test 예측 돌리기################

In [ ]:
test_stn_A = test_stn_A.fillna(0)

In [ ]:
test_stn_A = test_stn_A.drop(labels=['year','month','day','time','minute', 'season', 'stn_id', 'group', 'datetime','class'], axis=1)

In [ ]:
# test _ 학습된 개별 모델들이 각자 반환하는 예측 데이터 세트를 생성하고 개별 모델의 정확도 측정
knn_pred = knn_clf.predict(test_stn_A)
rf_pred = rf_clf.predict(test_stn_A)
dt_pred = dt_clf.predict(test_stn_A)
ada_pred = ada_clf.predict(test_stn_A)

# 기반 모델의 예측 결과를 스태킹
test_stacked_A = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(test_stacked_A.shape)

# transpose를 이용, 행과 열의 위치를 교환
test_stacked_A = np.transpose(test_stacked_A)
print(test_stacked_A.shape)

# 예측 및 성능 평가
pred_A = lr_final.predict(test_stacked_A)

In [ ]:
# 테스트 데이터프레임에 예측 결과 추가
test_stn_A['pred_A'] = pred_A

In [ ]:
test_stn_A['class'].value_counts()

## CV 세트 기반의 스태킹 앙상블

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수.
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')

    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]

        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장.
        test_pred[:, folder_counter] = model.predict(X_test_n)

    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)

    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

In [ ]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train_resampled, y_train_resampled, X_val, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train_resampled, y_train_resampled, X_val, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train_resampled, y_train_resampled, X_val,  7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train_resampled, y_train_resampled, X_val, 7)

In [ ]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)
print('원본 학습 피처 데이터 Shape:',X_train.shape, '원본 테스트 피처 Shape:',X_val.shape)
print('스태킹 학습 피처 데이터 Shape:', Stack_final_X_train.shape,
      '스태킹 테스트 피처 데이터 Shape:',Stack_final_X_test.shape)

In [ ]:
lr_final_cv = lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final_cv.predict(Stack_final_X_test)

mae = mean_absolute_error(y_val, stack_final)
accuracy = accuracy_score(y_val, stack_final)
f1 = f1_score(y_val, stack_final, average='weighted')

print(f"cv 기반의 스태킹 앙상블 모델의 MAE: {mae}")
print(f"cv 기반의 스태킹 앙상블 모델의 정확도: {accuracy}")
print(f"cv 기반의 스태킹 앙상블 모델의 F1 점수: {f1}")